## Creación de base de datos y tabla

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Date
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class MyTable(Base):
    __tablename__ = 'compras'
    id = Column(Integer, primary_key = True, autoincrement = True)
    id_usuario = Column(Integer)
    fecha = Column(Date)
    precio = Column(Integer)

engine = create_engine('sqlite:///Pragma.db')
Base.metadata.create_all(engine)


## Lectura de archivos y llenado de base de datos

In [69]:
def print_stat(conteo, promedio, minimo, maximo):
    stat = pd.DataFrame(dict(conteo=[conteo], promedio=[promedio], mínimo=[minimo], máximo=[maximo]))
    print(stat)
    print("\n")
    
import csv
import pandas as pd
  
LISTA_DE_ARCHIVOS = ["2012-1.csv"] #todo: completar lista

# todo: iterar a través de todos los archivos

fechas = []
precios = []
ids = []
f_casting_precios = lambda x: int(x) if len(x)> 0 else None

with open("Archivos/"+LISTA_DE_ARCHIVOS[0], "r") as archivo:
    lector = csv.reader(archivo, delimiter=",")
    next(lector)
    for linea in lector:
        fechas.append(linea[0])
        precios.append(f_casting_precios(linea[1]))
        ids.append(int(linea[2]))

# Load formated CSV data into a pandas dataframe
diccionario = dict(fecha=fechas, precio=precios, id_usuario=ids)
df = pd.DataFrame(diccionario)

# Set batch size
batch_size = 5
conteo = 0
promedio = None
minimo = 100
maximo = 0
calc_cum_promedio = lambda anterior_prom, prom: (anterior_prom+prom)/2 if anterior_prom is not None else prom
calc_cum_min = lambda anterior_min, min_: min_ if min_ < anterior_min else anterior_min
calc_cum_max = lambda anterior_max, max_: max_ if max_ > anterior_max else anterior_max

# Iterate over the dataframe in batches
batch_count = 1
for i in range(0, len(df), batch_size):
    # Extract batch of data
    batch = df[i:i+batch_size]
    # Write batch to SQL database
    try:
        batch.to_sql('compras', con=engine, if_exists='append', index=False)
        print("Cargue exitoso del bache Nº", batch_count, "/", batch_size)
        batch_count+=1
        conteo += batch.fecha.count()
        promedio = calc_cum_promedio(promedio,batch.precio.mean())
        minimo = calc_cum_min(minimo, batch.precio.min())
        maximo = calc_cum_max(maximo, batch.precio.max())
        print_stat(conteo, promedio, minimo, maximo)
    except Exception as e:
        print("Error mientras se guardaba la información en la base de datos:")
        print(e.args)
        


Cargue exitoso del bache Nº 1 / 5
   conteo  promedio  mínimo  máximo
0       5      47.0    14.0    87.0


Cargue exitoso del bache Nº 2 / 5
   conteo  promedio  mínimo  máximo
0      10      63.0    14.0    97.0


Cargue exitoso del bache Nº 3 / 5
   conteo  promedio  mínimo  máximo
0      15    54.875    14.0    97.0


Cargue exitoso del bache Nº 4 / 5
   conteo  promedio  mínimo  máximo
0      20   56.2375    14.0    97.0


Cargue exitoso del bache Nº 5 / 5
   conteo  promedio  mínimo  máximo
0      22  72.11875    14.0    97.0


